# Optimizing Equation Weights With Generic Algorithm

In [1]:
import numpy

## Genetic Algoritm Functions

In [13]:
def fitness(equation_inputs, population):
    """
    Calculate the fitness value of each solution in the current population.
    The fitness function calculates the sum of products between each input and its corresponding weight.
    """
    
    return numpy.sum(population * equation_inputs, axis=1)


def select_best_parents(population, fitness_values, n_parents):
    """
    Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    """
    parents = numpy.empty((n_parents, population.shape[1]))
    for parent in range(n_parents):
        max_fitness_idx = numpy.where(fitness_values == numpy.max(fitness_values))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent, :] = population[max_fitness_idx, :]
        fitness_values[max_fitness_idx] = -999999
    return parents

def cross_over(parents, offspring_size):
    """
    Cross over the parents and get offsprings
    """
    offspring = numpy.empty(offspring_size)
    #  The point at which crossover takes place between two parents. Usually it is at the center.
    cross_over_point = numpy.uint8(offspring_size[1]/2)
    
    for k in range(offspring_size[0]):
        parent_1_idx = k % parents.shape[0]         # In circular fashion
        parent_2_idx = (k+1) % parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent the part from the second parent. 
        offspring[k,0:cross_over_point] = parents[parent_1_idx, 0:cross_over_point]
        offspring[k,cross_over_point:] = parents[parent_2_idx, cross_over_point:]
    return offspring

def mutate(offspring_crossover):
    """
    Mutate a single gene in each offspring randomly.
    """
    for idx in range(offspring_crossover.shape[0]):
        # Add random value to a gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

The target is to maximize this equation ASAP: <br/>
y = w<sub>1</sub>x<sub>1</sub> + w<sub>2</sub>x<sub>2</sub> + w<sub>3</sub>x<sub>3</sub> + w<sub>4</sub>x<sub>4</sub> + w<sub>5</sub>x<sub>5</sub> + w<sub>6</sub>x<sub>6</sub>

where (x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, x<sub>4</sub>, x<sub>5</sub>, x<sub>6</sub>)=(4, -2, 3.5, 5, -11, -4.7)

We are going to use the genetic algorithm for the best possible values after a number of generations.

In [14]:
# Inputs of the equation.
equation_inputs = [4,-2,3.5,5,-11,-4.7]

# Number of the weights we are looking to optimize.
n_weights = 6

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
solution_per_population = 8
n_parents_mating = 4

# The population will have sol_per_pop chromosome where each chromosome has num_weights genes.
population_size = (solution_per_population, n_weights)

# Creating the initial population
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=population_size)

new_population

array([[ 3.49420988,  3.09351571, -3.50896215, -1.78786719,  2.32980372,
        -2.24482836],
       [-3.88567145,  2.03534298, -0.50832458,  1.18435314,  0.35062223,
        -2.86351271],
       [-2.11407151, -2.13662756,  1.66378843,  0.9423361 , -1.8888901 ,
        -2.16801647],
       [-1.78785967, -0.70598962,  0.18544499,  0.84444249,  2.03130726,
         3.78193581],
       [ 1.95329918,  2.25609673,  2.40907556,  1.17209093,  1.13868448,
        -0.38808826],
       [-1.38127768,  2.63354571, -0.89483159,  3.18486733,  3.40692298,
         3.4353701 ],
       [ 0.45139998, -1.28464859, -1.36076736, -1.97794406,  2.05130094,
        -3.8889692 ],
       [ 0.06920251,  2.1781542 , -0.13843026,  2.33248041, -3.5067344 ,
        -0.61754158]])

In [15]:
n_generations = 5

for generation in range(n_generations):
    print(f"Generation: {generation}")
    
    # Calculate fitness of each chromosome
    fitness_values = fitness(equation_inputs, new_population)
    
    # Select the best parents for mating
    best_parents = select_best_parents(new_population, fitness_values, n_parents_mating)
    
    # Generate next generation using cross-over
    offspring_crossover = cross_over(best_parents, offspring_size=(population_size[0] - best_parents.shape[0], n_weights))
    
    # Add some variations to offsprings using mutation.
    mutated_offsprings = mutate(offspring_crossover)
    
    # Create the new generation based on best parents and offsprings
    new_population[:best_parents.shape[0], :] = best_parents
    new_population[best_parents.shape[0]:, :] = mutated_offsprings
    
    # Print the best results of current iteration
    print("Best Result : ", numpy.max(numpy.sum(new_population*equation_inputs, axis=1)))

# Get best solutions after all iterations are done.
fitness_of_final_population = fitness(equation_inputs, new_population)

# Return the index of the best fit choromosome
best_match_idx = numpy.where(fitness_of_final_population == numpy.max(fitness_of_final_population))

print("Best Solution : ", new_population[best_match_idx, :])
print("Best Solution Fitness : ", fitness_of_final_population[best_match_idx])

Generation: 0
Best Result :  48.57492162697324
Generation: 1
Best Result :  69.55880363184643
Generation: 2
Best Result :  72.06557335022445
Generation: 3
Best Result :  77.1200811822613
Generation: 4
Best Result :  77.58671027338963
Best Solution :  [[[ 1.95329918  2.25609673  2.40907556  2.33248041 -4.66264501
   -0.61754158]]]
Best Solution Fitness :  [77.58671027]
